In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import sys
sys.path.append("../..")

In [3]:
import pandas as pd
import numpy as np
import pipit as pp

In [4]:
trace = pp.Trace.from_otf2("../../pipit/tests/data/ping-pong-otf2/")
pandas_df = trace.events[["Timestamp (ns)", "Name", "Event Type", "Process", "Thread"]].copy()
pandas_df["Process"] = pandas_df["Process"].astype(int)
idx = pandas_df.groupby(['Process']).cumcount()
pandas_df["idx"] = idx

In [5]:
pp.set_option("backend", "pandas")
pp.get_option("backend")

'pandas'

In [6]:
ds = pp.dsl2.MultiRankTrace()
ds.show()

MultiRankTrace (0 events)


In [8]:
# simulating a reader
row = pandas_df.iloc[0]
evt = pp.dsl.Event(
    rank=row["Process"],
    idx=row["idx"],
    thread=row["Thread"],
    timestamp=row["Timestamp (ns)"],
    event_type=row["Event Type"],
    name=row["Name"],
)
evt

:Event {'rank': 1, 'idx': 0, 'thread': 0, 'timestamp': 0.0, 'event_type': 'Instant', 'name': 'ProgramBegin'}

In [9]:
ds.push_event(evt)
ds.show()

MultiRankTrace (0 events)


In [10]:
ds.flush()
ds.show()

MultiRankTrace (1 events)


In [11]:
for i in range(1, len(pandas_df)):
    row = pandas_df.iloc[i]
    evt = pp.dsl.Event(
        rank=row["Process"],
        idx=row["idx"],
        thread=row["Thread"],
        timestamp=row["Timestamp (ns)"],
        event_type=row["Event Type"],
        name=row["Name"],
    )
    ds.push_event(evt)

ds.flush()
ds.show()

MultiRankTrace (120 events)


In [12]:
ds.filter("timestamp > 1e5").show()

AttributeError: 'NoneType' object has no attribute 'show'

In [13]:
ds.loc[0, 28]

TypeError: 'NoneType' object is not subscriptable